# HCI Aggregated for Health Data +

HCI Aggregated for Health Data +


In [1]:
from ambry import get_library
l = get_library()
b = l.bundle('cdph.ca.gov-hci-hdp-0.0.1')
w = b.warehouse('hci_counties')
#w.clean()
w.dsn

'sqlite://///Users/eric/proj/virt/ambry-develop/data/build/cdph.ca.gov/hci-hdp/warehouses/hci_counties.db'

In [2]:
w.query("""
CREATE VIEW gvids AS
SELECT DISTINCT gvid FROM census.gov-tiger-2015-counties
WHERE statefp = 6;

CREATE VIEW race_eths AS
SELECT DISTINCT race_eth_name FROM cdph.ca.gov-hci-poverty_rate-county;

CREATE VIEW race_gvid_index AS
SELECT * FROM gvids JOIN race_eths;

""").close()

In [3]:

w.query("""

-- Get only counties in California
CREATE VIEW geo AS SELECT gvid, name AS county_name,  geometry FROM census.gov-tiger-2015-counties
WHERE statefp = 6;

CREATE VIEW pr_overall AS 
SELECT gvid, race_eth_name, percent as poverty_percent, 
ll_95ci_percent as poverty_percent_l95,
ul_95ci_percent as poverty_percent_u95
FROM cdph.ca.gov-hci-poverty_rate-county
WHERE poverty = 'Overall' and reportyear ='2006-2010';

CREATE VIEW pr_child AS
SELECT gvid, race_eth_name, percent as  poverty_child_percent, 
ll_95ci_percent as poverty_child_percent_l95,
ul_95ci_percent as poverty_child_percent_u95
FROM cdph.ca.gov-hci-poverty_rate-county
WHERE poverty = 'Child' and reportyear ='2006-2010';;

CREATE VIEW pr_conc AS 
SELECT gvid, race_eth_name, percent as  poverty_concentrated_percent, 
ll_95ci_percent as poverty_concentrated_percent_l95,
ul_95ci_percent as poverty_concentrated_percent_u95
FROM cdph.ca.gov-hci-poverty_rate-county
WHERE poverty = 'Concentrated' and reportyear ='2006-2010';;

CREATE VIEW hf_total AS SELECT gvid,  race_eth_name, mrfei FROM cdph.ca.gov-hci-healthy_food-county;

""").close()

In [4]:
# 
# LEFT JOIN pr_conc AS prco on prco.gvid = pro.gvid AND prco.race_eth_name = pro.race_eth_name

poverty_df = w.dataframe("""
SELECT * FROM race_gvid_index AS idx
LEFT JOIN census.gov-tiger-2015-counties AS geo ON idx.gvid = geo.gvid
LEFT JOIN pr_overall AS pro on pro.gvid = geo.gvid
LEFT JOIN pr_child AS prch on prch.gvid = pro.gvid AND prch.race_eth_name = pro.race_eth_name
LEFT JOIN pr_conc AS prco on prco.gvid = pro.gvid AND prco.race_eth_name = pro.race_eth_name
LEFT JOIN hf_total as hfh ON hfh.gvid = pro.gvid AND hfh.race_eth_name = pro.race_eth_name 
;
""")


In [61]:
# Seperately query the unsafe water table for each of the categories, 
# Then join them in to a single table. 

wq_query_template = """
SELECT gvid, race_eth_name, percent AS {category}_percent
FROM cdph.ca.gov-hci-unsafe_water-county
WHERE category = "{category}"
"""

water_df  = w.dataframe('SELECT * FROM race_gvid_index').set_index(['gvid','race_eth_name'])

# Get all of the categories. 
for category in w.dataframe('SELECT DISTINCT category FROM cdph.ca.gov-hci-unsafe_water-county').category:

    print category
    
    q = wq_query_template.format(category=category)
    
    water_df = water_df.join( w.dataframe(q).set_index(['gvid','race_eth_name']))
    

arsenic
nitrates
other
tcrb
total
tt


In [6]:

# Slice across counties to get all White race/eth
# water_df.xs('White', level =1 )

# Slice across race/eth to get aa race/eth for counties
# water_df.xs('0O060r', level = 0 )


In [62]:

poverty_query_template = """
SELECT gvid, race_eth_name, percent as poverty_{categorylc}_percent, 
ll_95ci_percent as poverty_{categorylc}_percent_l95,
ul_95ci_percent as poverty_{categorylc}_percent_u95
FROM  cdph.ca.gov-hci-poverty_rate-county
WHERE poverty = "{category}" and reportyear ="2006-2010"
"""

poverty_df = w.dataframe('SELECT * FROM race_gvid_index').set_index(['gvid','race_eth_name'])

# Get all of the categories. 
for category in w.dataframe('SELECT DISTINCT poverty FROM cdph.ca.gov-hci-poverty_rate-county').poverty:

    if not category:
        continue
        
    print category
    
    q = poverty_query_template.format(category=category, categorylc=category.lower())

    poverty_df = poverty_df.join(w.dataframe(q).set_index(['gvid','race_eth_name']))

Child
Concentrated
Overall
